In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import seaborn as sns 
from collections import defaultdict
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
df = pd.read_csv('../data/final-data.csv', sep='~')
x = range(3,len(df.columns)-1)
df = df.iloc[:,x]
df['rym rating'] = df['rym rating']*20
# p and (1-p ) hyperparameter tuning
df['rymspot rate'] = (df['rym rating']*.5 + df['popularity']*.5)
df['rymspot rate'].describe()

count    7252.000000
mean       56.457750
std         9.763336
min        37.600000
25%        49.300000
50%        56.800000
75%        63.200000
max        85.700000
Name: rymspot rate, dtype: float64

In [42]:
labels = []
for d in df['rymspot rate']:
    labels+=[0 if d >= 75 else 1 if d >= 50 else 2]

In [43]:
x = range(3,len(df.columns)-17)
X = df.iloc[:,x]
X

,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,-14.074,0,0.0991,0.90700,0.000004,0.1110,0.377,91.240
1,-14.650,1,0.0704,0.89500,0.000000,0.1110,0.459,141.231
2,-12.273,1,0.0841,0.87600,0.000001,0.1040,0.480,152.197
3,-12.577,1,0.0620,0.79400,0.000000,0.1470,0.483,145.376
4,-11.647,0,0.2490,0.84400,0.000005,0.1120,0.175,64.841
...,...,...,...,...,...,...,...,...
7247,-9.650,1,0.0443,0.00724,0.004210,0.1910,0.775,147.919
7248,-10.027,1,0.0803,0.62800,0.464000,0.0622,0.343,99.975
7249,-12.967,1,0.0284,0.86500,0.684000,0.0876,0.556,86.995
7250,-10.179,1,0.0294,0.57500,0.878000,0.0720,0.394,144.118


In [44]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
scores = []
for x in range(1,20):
    X_train, X_test, y_train,y_test =train_test_split(
                                         X, labels, test_size = 0.2)
    gnb = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    naive = gnb.fit(X_train,y_train)
    y_pred = naive.predict(X_test)

    scores += [accuracy_score(y_test, y_pred)] 
    
print(sum(scores)/len(scores))

0.7380028292647539


In [49]:
from sklearn.naive_bayes import GaussianNB
scores = []
for x in range(1,20):
    X_train, X_test, y_train,y_test =train_test_split(
                                         X, labels, test_size = 0.2)
    gnb = GaussianNB()
    naive = gnb.fit(X_train,y_train)
    y_pred = naive.predict(X_test)

    scores += [accuracy_score(y_test, y_pred)]
print(sum(scores)/len(scores))

[0.72915231 0.70778773 0.70965517 0.74689655 0.72758621]
0.7160941637346295


In [11]:
from sklearn import tree

scores = []
for i in range(1,20):
    X_train, X_test, y_train,y_test =train_test_split(
                                             X, labels, test_size = 0.2)
    for x in range(1,100):
        gnb = tree.DecisionTreeClassifier(random_state = 0, max_depth=i)
        naive = gnb.fit(X_train,y_train)
        y_pred = naive.predict(X_test)

        scores += [accuracy_score(y_test, y_pred)]

    print(sum(scores)/len(scores))

0.7029634734665742
0.7174362508614781
0.7137606248564218
0.7196760854583085
0.7235010337698141
0.7253618194348743
0.7260017721768282
0.7263094417643091
0.7246343517880482
0.7228118538938771
0.7231376480170516
0.7216861934298285
0.7201399565286659
0.719700699025311
0.7167930163106015
0.7145072363887025
0.7111930919852418
0.7097021211425057
0.706228009721064


In [51]:
from sklearn import tree
scores = []
import xgboost as xgb
from sklearn.model_selection import cross_val_score
# X_train, X_test, y_train,y_test =train_test_split(
#                                          X, labels, test_size = 0.2)
print(cross_val_score(xgb.XGBClassifier(), X,labels) )
scores2 = []
for x in range(1,100):
    
    X_train, X_test, y_train,y_test =train_test_split(
                                         X, labels, test_size = 0.2)
    gnb = tree.DecisionTreeClassifier(random_state = 0, max_depth=7)
    naive = gnb.fit(X_train,y_train)
    boost = xgb.XGBClassifier()
    bo = boost.fit(X_train,y_train)
    y_pred = naive.predict(X_test)
    y_pred2 = bo.predict(X_test)
    scores += [accuracy_score(y_test, y_pred)]
    scores2 += [accuracy_score(y_test,y_pred2)]

print(sum(scores)/len(scores))

[0.72915231 0.70778773 0.70965517 0.74689655 0.72758621]
0.7283308620317579


In [52]:
print(sum(scores2)/len(scores2))

0.747189329546325


In [13]:
from sklearn.neighbors import KNeighborsClassifier

scores = []
for i in range(18,28):
    X_train, X_test, y_train,y_test =train_test_split(
                                             X, labels, test_size = 0.2)
    for x in range(1,50):
        gnb = KNeighborsClassifier(n_neighbors=i)
        naive = gnb.fit(X_train,y_train)
        y_pred = naive.predict(X_test)

        scores += [accuracy_score(y_test, y_pred)]

    print(sum(scores)/len(scores))
#     19 nn

0.7188146106133706
0.7177808407994491
0.7172065242361613
0.7172639558924914
0.7192281185389391
0.7231794164943702
0.7211775130451892
0.7204514128187435
0.7191209127804573
0.7188835286009664


In [14]:
from sklearn.neural_network import MLPClassifier
scores = []
for x in range(1,100):
    X_train, X_test, y_train,y_test =train_test_split(
                                         X, labels, test_size = 0.2)
    gnb = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
    naive = gnb.fit(X_train,y_train)
    y_pred = naive.predict(X_test)

    scores += [accuracy_score(y_test, y_pred)]

print(sum(scores)/len(scores))
#     19 nn

0.704696865275776


In [15]:
# %pip install surprise

In [16]:
# from surprise import KNNWithMeans
# from surprise import Dataset
# from surprise import accuracy
# from surprise import Reader
# from surprise.model_selection import train_test_split



In [17]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

# import seaborn as sns 
# from collections import defaultdict
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
# df = pd.read_csv('../data/final-data.csv', sep='~')
# x = range(3,len(df.columns)-2)
# df = df.iloc[:,x]
# df['rym rating'] = df['rym rating']*20
# df['Rating'] = (df['rym rating']*.5 + df['popularity']*.5)
# df['Rating'].describe()
# df = df.iloc[:,[12,13,-1]]
# df
# len(np.unique(df.iloc[:,0]))

In [18]:
# reader = Reader(rating_scale=(1, 100))
# data = Dataset.load_from_df(df,reader)

In [19]:
# trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

In [20]:
# algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
# algo.fit(trainset)

In [21]:
# test_pred = algo.test(testset)

In [22]:
# test_pred

In [23]:
# print("Item-based Model : Test Set")
# accuracy.rmse(test_pred, verbose=True)

https://www.kaggle.com/code/saurav9786/recommender-system-using-amazon-reviews/notebook 

In [24]:
df = pd.read_csv('../data/final-data.csv', sep='~')
x = range(3,len(df.columns)-1)
df = df.iloc[:,x]
x = list(range(3,len(df.columns)-16))
x+=[14,-1]
X = df.iloc[:,x]
X

,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_href,sp_genres
0,-14.074,0,0.0991,0.90700,0.000004,0.1110,0.377,91.240,https://api.spotify.com/v1/tracks/5sQGo3fB2NoF...,"['cantautor', 'latin alternative', 'nueva canc..."
1,-14.650,1,0.0704,0.89500,0.000000,0.1110,0.459,141.231,https://api.spotify.com/v1/tracks/1w6mcMacExya...,"['cantautor', 'latin alternative', 'nueva canc..."
2,-12.273,1,0.0841,0.87600,0.000001,0.1040,0.480,152.197,https://api.spotify.com/v1/tracks/33QDwXW1O0hA...,"['cantautor', 'latin alternative', 'nueva canc..."
3,-12.577,1,0.0620,0.79400,0.000000,0.1470,0.483,145.376,https://api.spotify.com/v1/tracks/2ja41V8fXBHR...,"['cantautor', 'latin alternative', 'nueva canc..."
4,-11.647,0,0.2490,0.84400,0.000005,0.1120,0.175,64.841,https://api.spotify.com/v1/tracks/2hf6vQr1Ogju...,"['cantautor', 'latin alternative', 'nueva canc..."
...,...,...,...,...,...,...,...,...,...,...
7247,-9.650,1,0.0443,0.00724,0.004210,0.1910,0.775,147.919,https://api.spotify.com/v1/tracks/7BpqssfH7oRR...,"['classic j-rock', 'j-rock', 'japanese dream p..."
7248,-10.027,1,0.0803,0.62800,0.464000,0.0622,0.343,99.975,https://api.spotify.com/v1/tracks/682TDtMAq9Uc...,"['classic j-rock', 'j-rock', 'japanese dream p..."
7249,-12.967,1,0.0284,0.86500,0.684000,0.0876,0.556,86.995,https://api.spotify.com/v1/tracks/0msOgZw6XpjS...,"['classic j-rock', 'j-rock', 'japanese dream p..."
7250,-10.179,1,0.0294,0.57500,0.878000,0.0720,0.394,144.118,https://api.spotify.com/v1/tracks/2yIywMeXB1A4...,"['classic j-rock', 'j-rock', 'japanese dream p..."


In [25]:
import string
for r in range(len(X['sp_genres'])):
    X.iloc[r,-1] = X.iloc[r,[-1]]['sp_genres'].translate(str.maketrans('', '', string.punctuation))
X



,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_href,sp_genres
0,-14.074,0,0.0991,0.90700,0.000004,0.1110,0.377,91.240,https://api.spotify.com/v1/tracks/5sQGo3fB2NoF...,cantautor latin alternative nueva cancion rock...
1,-14.650,1,0.0704,0.89500,0.000000,0.1110,0.459,141.231,https://api.spotify.com/v1/tracks/1w6mcMacExya...,cantautor latin alternative nueva cancion rock...
2,-12.273,1,0.0841,0.87600,0.000001,0.1040,0.480,152.197,https://api.spotify.com/v1/tracks/33QDwXW1O0hA...,cantautor latin alternative nueva cancion rock...
3,-12.577,1,0.0620,0.79400,0.000000,0.1470,0.483,145.376,https://api.spotify.com/v1/tracks/2ja41V8fXBHR...,cantautor latin alternative nueva cancion rock...
4,-11.647,0,0.2490,0.84400,0.000005,0.1120,0.175,64.841,https://api.spotify.com/v1/tracks/2hf6vQr1Ogju...,cantautor latin alternative nueva cancion rock...
...,...,...,...,...,...,...,...,...,...,...
7247,-9.650,1,0.0443,0.00724,0.004210,0.1910,0.775,147.919,https://api.spotify.com/v1/tracks/7BpqssfH7oRR...,classic jrock jrock japanese dream pop shibuyakei
7248,-10.027,1,0.0803,0.62800,0.464000,0.0622,0.343,99.975,https://api.spotify.com/v1/tracks/682TDtMAq9Uc...,classic jrock jrock japanese dream pop shibuyakei
7249,-12.967,1,0.0284,0.86500,0.684000,0.0876,0.556,86.995,https://api.spotify.com/v1/tracks/0msOgZw6XpjS...,classic jrock jrock japanese dream pop shibuyakei
7250,-10.179,1,0.0294,0.57500,0.878000,0.0720,0.394,144.118,https://api.spotify.com/v1/tracks/2yIywMeXB1A4...,classic jrock jrock japanese dream pop shibuyakei


In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["sp_genres"])
# print("Count Matrix:", count_matrix.toarray())
# np.shape(count_matrix)
# cosine_sim = cosine_similarity(count_matrix)
arr2 = []
np.asarray(arr2)
arr = count_matrix.toarray()
arr.astype(float)
print(len(arr))
for i in range(len(arr)):
    temp = []
    for x in range(len(X.columns)-2):
        temp += [X.iloc[i,x]]
    arr2 += [arr[i].tolist() + temp]
# len(arr2)
cosine_sim = cosine_similarity(arr2)

7252


In [28]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["sp_genres"])

arr2 = []
for i in range(7252):
    temp = []
    for x in range(len(X.columns)-2):
        temp += [X.iloc[i,x]]
    arr2 += [ temp]
# len(arr2)
cosine_sim = cosine_similarity(arr2)
cosine_sim

array([[1.        , 0.99873446, 0.99733688, ..., 0.99988945, 0.99653537,
        0.99625676],
       [0.99873446, 1.        , 0.99973762, ..., 0.99895499, 0.99943921,
        0.99927708],
       [0.99733688, 0.99973762, 1.        , ..., 0.99766783, 0.99993024,
        0.99984684],
       ...,
       [0.99988945, 0.99895499, 0.99766783, ..., 1.        , 0.99696765,
        0.99664776],
       [0.99653537, 0.99943921, 0.99993024, ..., 0.99696765, 1.        ,
        0.99995693],
       [0.99625676, 0.99927708, 0.99984684, ..., 0.99664776, 0.99995693,
        1.        ]])

In [29]:
df.name

0                                                   Ojalá
1                              Óleo de Mujer Con Sombrero
2                                     Canción Del Elegido
3                       Al Final de Este Viaje en la Vida
4              La Familia, la Propiedad Privada y el Amor
                              ...                        
7247                                           MAGIC LOVE
7248                                       WEATHER REPORT
7249                                        IN THE FLIGHT
7250                                         バックビートにのっかって
7251    Sound Test (From "Shin Megami Tensei III Noctu...
Name: name, Length: 7252, dtype: object

In [30]:
namelike = 'Silent All These Years - 2015 Remaster'
def get_index_from_name(name):
    return df.index[df['name'] == name].tolist()

idx = get_index_from_name(namelike)

similar_songs = list(enumerate(cosine_sim[idx]))
songs = np.argsort(similar_songs[0][1])
topfive = songs[-2:-7:-1]
for i in range(len(topfive)):
    print(i+1,df.iloc[topfive[i]]['name'])



1 The Ruins
2 Cantores Que Reflexionan
3 Linden Arden Stole the Highlights
4 The Damned Don't Cry
5 Have Mercy on Me (Live) [feat. David Gungor]


In [31]:

# df2 = pd.read_csv('../data/final-data.csv', sep='~')
# x = range(3,len(df2.columns)-1)
# df2 = df2.iloc[:,x]
# df2['rym rating'] = df2['rym rating']*20
# df2['rymspot rate'] = (df2['rym rating']*.5 + df2['popularity']*.5)
# # df['rymspot rate'] = df['popularity']
# x = range(3,len(df.columns)-17)
# X2 = df.iloc[:,x]
# labels = []

# for d in df2['rymspot rate']:
#     labels+=[0 if d >= 75 else 1 if d >= 50 else 2]

# dftest = df.iloc[topfifteen]
# x = range(3,len(dftest.columns)-17)
# dftest = dftest.iloc[:,x]

# X2  = X2.drop(topfifteen)
# top = topfifteen[::-1]
# for i in top:
#     del labels[i]

# # gnb = tree.DecisionTreeClassifier(random_state = 0, max_depth=7)
# # naive = gnb.fit(X2,labels)
# # y_pred = naive.predict(X_test)
# # len(y_pred)

In [32]:
df2 = pd.DataFrame(similar_songs[0][1])

df2.describe()

,0
count,7252.000000
mean,0.996480
std,0.014267
min,0.084550
25%,0.998096
50%,0.999383
75%,0.999826
max,1.000000


In [33]:
df2

,0
0,0.997610
1,0.999819
2,0.999990
3,0.999996
4,0.995597
...,...
7247,0.999783
7248,0.999865
7249,0.997933
7250,0.999879


https://towardsdatascience.com/using-cosine-similarity-to-build-a-movie-recommendation-system-ae7f20842599

NameError: name 'Xtrain' is not defined